## <center>Regularization with SciKit-Learn</center>

Previously we created a new polynomial feature set and then applied our standard linear regression on it, but we can be smarter about model choice and utilize regularization.

Regularization attempts to minimize the RSS (residual sum of squares) *and* a penalty factor. This penalty factor will penalize models that have coefficients that are too large. Some methods of regularization will actually cause non useful features to have a coefficient of zero, in which case the model does not consider the feature.

Let's explore two methods of regularization, Ridge Regression and Lasso. We'll combine these with the polynomial feature set (it wouldn't be as effective to perform regularization of a model on such a small original feature set of the original X).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Data and Setup

In [2]:
df = pd.read_csv("C:/Users/Lenovo/Desktop/Python/Machine Learning/Supervised Learning/Linear regression/Data Sets/Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

#### Polynomial Conversion

In [3]:
from sklearn.preprocessing import PolynomialFeatures

In [4]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)
poly_features = polynomial_converter.fit_transform(X)

#### Train, Test and Split

In [5]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=42)

#### Scaling the Data

While our particular data set has all the values in the same order of magnitude ($1000s of dollars spent), typically that won't be the case on a dataset, and since the mathematics behind regularized models will sum coefficients together, its important to standardize the features. Review the theory videos for more info, as well as a discussion on why we only **fit** to the training data, and **transform** on both sets separately.

In [8]:
from sklearn.preprocessing import StandardScaler

Standard Scaler uses a Z-scaler:

$$X_{scaled} = \frac{X-\bar{X}}{\sigma}$$

In [9]:
scaler = StandardScaler()
scaler.fit(X_train)  # To avoid data leakage, the data set is only fitted to the training set
                     # y_train or y_test are not needed to be scaled.

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Ridge Regression

Ridge regression is also a linear model for regression,  so the formula it uses to make predictions is the same one used for ordinary least squares. In ridge regression, though, the coefficients (w) are chosen not only so that they predict well on the training data, but also to fit an additional constraint. We also want the magnitude of coefficients to be as small as possible; in other words, all entries of w should be close to zero. Intuitively, this means each feature should have as little effect on the outcome as possible (which translates to having a small slope), while still predicting well. This constraint is an example of what is called regularization. Regularization means explicitly restricting a model to avoid overfitting. The particular kind used by ridge regression is known as L2 regularization.

#Ridge regression minimizes the objective function:

$$Error = \sum_{i=1}^{n} (y_i - \beta_0 - \sum_{j=1}^{p} \beta_{j}x_{ij})^2 + \lambda \sum_{j=1}^{p} \beta_j^2$$

This model solves a regression model where the loss function is the linear least squares function and regularization is given by the l2-norm. Also known as Ridge Regression or Tikhonov regularization.

In [11]:
from sklearn.linear_model import Ridge

In [12]:
ridge_model = Ridge(alpha=10)
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [19]:
y_pred = ridge_model.predict(X_test)

In [15]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [20]:
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
RMSE = np.sqrt(MSE)

In [17]:
MAE

0.6296591346758597

In [18]:
RMSE

0.8916327541710891

How did it perform on the training set? (This will be used later on for comparison)

In [21]:
# Training Set Performance
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.5230070613645759

The Ridge model makes a trade-off between the simplicity of the model (near-zero coefficients) and its performance on the training set. How much importance the model places on simplicity versus training set performance can be specified by the user, using the alpha parameter. Increasing alpha forces coefficients to move more toward zero, which decreases training set performance but might help generalization. For very small values of alpha, coefficients are barely restricted at all, and we end up with a model that resembles LinearRegression


### Choosing an alpha value with Cross-Validation

Review the theory video for full details.

In [22]:
from sklearn.linear_model import RidgeCV

In [23]:
# Choosing a scoring: https://scikit-learn.org/stable/modules/model_evaluation.html
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0), scoring='neg_mean_absolute_error')

# Important cv is set to None by default, which uses the take one out method. For large datasets, it's important 
# # to specify the value of cv, otherwise it will use many computer resources. 

# The more alpha options you pass, the longer this will take.
ridge_cv_model.fit(X_train,y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [24]:
ridge_cv_model.alpha_

0.1

In [25]:
y_pred_cv = ridge_cv_model.predict(X_test)

In [26]:
MAE = mean_absolute_error(y_test,y_pred_cv)
MSE = mean_squared_error(y_test,y_pred_cv)
RMSE = np.sqrt(MSE)

In [27]:
MAE

0.46671241131523317

In [28]:
RMSE

0.5945136671816592

In [29]:
# Training Set Performance
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.3240172629005777

In [30]:
ridge_cv_model.coef_

array([ 5.90523815,  0.46316396,  0.68028713, -6.17743395,  3.73671928,
       -1.40708382,  0.00624704,  0.11128917, -0.2617823 ,  2.17135744,
       -0.51480159,  0.70587211,  0.60311504, -0.53271216,  0.5716495 ,
       -0.34685826,  0.36744388, -0.03938079, -0.12192939])


-----

## Lasso Regression

In [38]:
from sklearn.linear_model import LassoCV

In [39]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
lasso_cv_model = LassoCV(eps=0.1,n_alphas=100,cv=5)

In [40]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.1)

In [41]:
lasso_cv_model.alpha_

0.4943070909225832

In [42]:
test_predictions = lasso_cv_model.predict(X_test)

In [43]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [44]:
MAE

0.6541723161252867

In [45]:
RMSE

1.1308001022762548

In [46]:
# Training Set Performance
# Training Set Performance
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.6912807140820709

In [47]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

## Elastic Net

Elastic Net combines the penalties of ridge regression and lasso in an attempt to get the best of both worlds!

In [48]:
from sklearn.linear_model import ElasticNetCV

In [51]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],tol=0.01)

In [52]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [55]:
elastic_model.l1_ratio_

1.0

In [56]:
test_predictions = elastic_model.predict(X_test)

In [57]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [58]:
MAE

0.5663262117569452

In [59]:
RMSE

0.7485546215633726

In [60]:
# Training Set Performance
# Training Set Performance
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.43075829904723684

In [61]:
elastic_model.coef_

array([ 3.78993643,  0.89232919,  0.28765395, -1.01843566,  2.15516144,
       -0.3567547 , -0.271502  ,  0.09741081,  0.        , -1.05563151,
        0.2362506 ,  0.07980911,  1.26170778,  0.01464706,  0.00462336,
       -0.39986069,  0.        ,  0.        , -0.05343757])

-----
---